In [2]:
df.info() # Gives concise summary of dataset
df.head(n) # Gives first n rows of dataset
df.describe() # Gives descriptives
df.describe(include = 'all') # Gives descriptives on all columns incl non INT
df_test = df[['body-style','price']] # and...
ef_grp = df_test.groupby(['body-style'], as_index = false).mean() # Gives averages grouped by body style
df['body-style'] # Accesses column 'body-style' from dataframe
df.rename(columns = {'city': 'Cities'}, inplace = True) # Renames column 'city' to 'Cities'

NameError: name 'df' is not defined

In [ ]:
# Linear model

from sklearn.linear_model import LinearRegression lm=LinearRegression()
X = df[['highway-mpg']]
Y = df['price']
lm.fit(X, Y)
Yhat=lm.predict(X)

In [ ]:
# Coefficient of determination

lm = LinearRegression()
X = df[['highway-mpg']]
Y = df['price']
lm.fit(X, Y)
out=lm.score(X,Y)

In [ ]:
# Split into train-test groups

train_test_split()

In [ ]:
# Split data into n folds for cross validation return x on y predictions for all three folds /
# Concatenate, each y getting a new prediction for an x it didn't train on

cross_val_predict (lr2e, x_data, y_data, cv=n)

In [ ]:
# Full train-tests with visualization

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

# --- Python Cheat Sheet for Train-Test Split and Model Evaluation ---

# Assuming you have a pandas DataFrame named 'df' with your variables X and Y

# 1. Define X (features) and Y (target)
#    Replace 'feature_column_1', 'feature_column_2', and 'target_column' with your actual column names
X = df[['feature_column_1', 'feature_column_2']] # Example for multiple features
Y = df['target_column'] # Your target variable

# If X is a single feature, it's good practice to reshape it to a 2D array
# X = df[['single_feature_column']]

# 2. Train-Test Split
#    test_size: Proportion of the dataset to include in the test split (e.g., 0.2 for 20%)
#    random_state: Ensures reproducibility of your split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")

# 3. Basic Linear Regression Model (as an example)
model = LinearRegression()
model.fit(X_train, Y_train)

# Make predictions
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse_train = mean_squared_error(Y_train, Y_train_pred)
mse_test = mean_squared_error(Y_test, Y_test_pred)

print(f"\n--- Basic Linear Regression Results ---")
print(f"Train MSE: {mse_train:.4f}")
print(f"Test MSE: {mse_test:.4f}")

# 4. Alpha Tests for Over/Underfitting (using MSE comparison)

print(f"\n--- Alpha Tests for Over/Underfitting ---")
# Rule of Thumb:
# If Train MSE is significantly lower than Test MSE: Potential Overfitting
# If both Train MSE and Test MSE are high and similar: Potential Underfitting
# If both Train MSE and Test MSE are low and similar: Good Fit

# Define a threshold for "significant difference" (can be adjusted)
threshold = 0.1 * mse_train # Example: 10% difference from train MSE

if abs(mse_train - mse_test) > threshold and mse_train < mse_test:
    print("WARNING: Potential Overfitting! Train MSE is significantly lower than Test MSE.")
elif mse_train > (mse_test * 1.5) and mse_test > np.mean(Y_train)**2: # Heuristic for high MSE
    print("WARNING: Potential Underfitting! Both Train and Test MSE are high, and relatively similar.")
else:
    print("Model appears to have a reasonable fit (neither severely overfit nor underfit).")
print(f"MSE Difference (Train - Test): {mse_train - mse_test:.4f}")


# 5. MSE/Polynomial-Order Graph for Determining Best Fit

# This section assumes X is a single feature for visualization purposes.
# If X has multiple features, you might want to perform dimensionality reduction
# or plot against a single dominant feature.

if X.shape[1] > 1:
    print("\nNote: For the Polynomial Order Graph, it's best suited for single features.")
    print("Consider using a single dominant feature from X or dimensionality reduction.")
    # For demonstration, we'll pick the first feature if multiple exist.
    X_single_feature = X.iloc[:, 0].values.reshape(-1, 1)
    X_train_poly, X_test_poly, Y_train_poly, Y_test_poly = train_test_split(
        X_single_feature, Y, test_size=0.2, random_state=42
    )
else:
    X_train_poly, X_test_poly, Y_train_poly, Y_test_poly = X_train, X_test, Y_train, Y_test


max_degree = 10 # Maximum polynomial degree to test
train_mse_errors = []
test_mse_errors = []
polynomial_degrees = range(1, max_degree + 1)

for degree in polynomial_degrees:
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly_transformed = poly_features.fit_transform(X_train_poly)
    X_test_poly_transformed = poly_features.transform(X_test_poly)

    poly_model = LinearRegression()
    poly_model.fit(X_train_poly_transformed, Y_train_poly)

    Y_train_pred_poly = poly_model.predict(X_train_poly_transformed)
    Y_test_pred_poly = poly_model.predict(X_test_poly_transformed)

    train_mse_errors.append(mean_squared_error(Y_train_poly, Y_train_pred_poly))
    test_mse_errors.append(mean_squared_error(Y_test_poly, Y_test_pred_poly))

# Plotting the MSE vs. Polynomial Order
plt.figure(figsize=(10, 6))
plt.plot(polynomial_degrees, train_mse_errors, label='Train MSE', marker='o')
plt.plot(polynomial_degrees, test_mse_errors, label='Test MSE', marker='o')
plt.xlabel('Polynomial Order')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('MSE vs. Polynomial Order to Determine Best Fit')
plt.xticks(polynomial_degrees)
plt.grid(True)
plt.legend()
plt.axvline(x=np.argmin(test_mse_errors) + 1, color='red', linestyle='--', label='Optimal Test MSE')
plt.text(np.argmin(test_mse_errors) + 1 + 0.2, min(test_mse_errors),
         f'Best Degree: {np.argmin(test_mse_errors) + 1}', color='red')
plt.show()

print(f"\n--- Polynomial Order Analysis ---")
optimal_degree_index = np.argmin(test_mse_errors)
optimal_degree = polynomial_degrees[optimal_degree_index]
print(f"The polynomial order with the lowest Test MSE is: {optimal_degree}")
print(f"Corresponding Train MSE: {train_mse_errors[optimal_degree_index]:.4f}")
print(f"Corresponding Test MSE: {test_mse_errors[optimal_degree_index]:.4f}")

# Re-evaluating alpha tests for the optimal polynomial model
if abs(train_mse_errors[optimal_degree_index] - test_mse_errors[optimal_degree_index]) > threshold and \
   train_mse_errors[optimal_degree_index] < test_mse_errors[optimal_degree_index]:
    print("WARNING: Potential Overfitting with optimal polynomial degree.")
elif train_mse_errors[optimal_degree_index] > (test_mse_errors[optimal_degree_index] * 1.5) and \
     test_mse_errors[optimal_degree_index] > np.mean(Y_train_poly)**2:
    print("WARNING: Potential Underfitting with optimal polynomial degree.")
else:
    print("Optimal polynomial model appears to have a reasonable fit.")